In [44]:
#imports
import numpy as np
import sys

In [45]:
#load small input
input = open("./../../data/day_11/small_input.txt", "r").readlines()

In [51]:
#load input
input = open("./../../data/day_11/input.txt", "r").readlines()

In [52]:
#process input
image = np.zeros((len(input), len(input[0].strip())), dtype=int)

for i, line in enumerate(input):
    line = line.strip()
    for j, char in enumerate(line):
        if char == "#":
            image[i,j] = 1
        elif char == ".":
            image[i,j] = 0
        else:
            print(f"Unexpected character {char} in image")

In [40]:
#expand
def expand(image):
    empty_rows = 0
    for row in range(image.shape[0]):
        if np.sum(image[row,:]) == 0:
            empty_rows += 1
    empty_cols = 0
    for col in range(image.shape[1]):
        if np.sum(image[:,col]) == 0:
            empty_cols += 1
    
    new_image = np.zeros((image.shape[0]+empty_rows, image.shape[1]+empty_cols), dtype=int)

    #print(f"size of original image is {image.shape}, size of new image is {new_image.shape}")

    offset_row = 0
    for row in range(image.shape[0]):
        offset_col = 0
        for col in range(image.shape[1]):
            #print(f"Processing position {(row,col)} with current offsets {(offset_row,offset_col)}")
            
            if np.sum(image[row,:]) == 0 and np.sum(image[col,:]) == 0:
                #both row and col are empty
                new_image[row+offset_row, col+offset_col] = 0
                if col == 0:
                    offset_row += 1
                offset_col += 1
                new_image[row+offset_row, col+offset_col] = 0
            elif np.sum(image[row,:]) == 0:
                #only row is empty
                new_image[row+offset_row, col+offset_col] = 0
                if col == 0:
                    offset_row += 1
                new_image[row+offset_row, col+offset_col] = 0
            elif np.sum(image[:,col]) == 0:
                #only col is empty
                new_image[row+offset_row, col+offset_col] = 0
                offset_col += 1
                new_image[row+offset_row, col+offset_col] = 0
            else:
                #neither is empty
                new_image[row+offset_row, col+offset_col] = image[row,col] #should be fine to only actually access image here? because if the entire row or col is 0, then image[row,col] also needs to be 0
    return new_image

In [41]:
image_expanded = expand(image)

In [33]:
def get_neighbours(pos,matrix_shape):
    x,y = pos
    xm,ym = matrix_shape
    neighbours = []
    if x > 0:
        neighbours.append((x-1,y))
    if y > 0:
        neighbours.append((x,y-1))
    if x < xm-1:
        neighbours.append((x+1,y))
    if y < ym-1:
        neighbours.append((x,y+1))
    return neighbours

def floyd_warshall(image):
    n = image.shape[0]*image.shape[1]
    D = np.zeros((n,n), dtype=int)
    D[:,:] = 1e8
    for i in range(1,n):
        xi,yi = i//image.shape[1],i%image.shape[1]
        for x,y in get_neighbours((xi,yi), image.shape):
            n2 = x*image.shape[0]+y
            D[i,n2] = 1
            D[n2,i] = 1
            
    for k in range(1, n):
        for i in range(1,n):
            for j in range(1,n):
                xk,yk = k//image.shape[1],k%image.shape[1]
                xi,yi = i//image.shape[1],i%image.shape[1]
                xj,yj = j//image.shape[1],j%image.shape[1]

                D[i,j] = min(D[i,j], D[i,k]+D[k,j])

    return D


In [34]:
# part 1
galaxies = []
for i in range(image_expanded.shape[0]):
    for j in range(image_expanded.shape[1]):
        if image_expanded[i,j] == 0:
            galaxies.append((i,j))

D = floyd_warshall(image_expanded)
galaxy_distances = []
for i,(g1x,g1y) in enumerate(galaxies):
    if i+1 < len(galaxies):
        for j,(g2x,g2y) in enumerate(galaxies[i+1:]):
            n1 = g1x*image_expanded[0]+g1y
            n2 = g2x*image_expanded[0]+g2y
            galaxy_distances.append(D[n1,n2])

summed_distances = sum(galaxy_distances)
print(f"Part 1: The sum of galaxy distances is {summed_distances}")


Part 1: The sum of galaxy distances is [1651008983 1651008983 1651008983 1651008983  515125532 1651008983
 1651008983 1651008983 1651008983 1651008983 1651008983 1651008983
 1651008983]


In [42]:
#floyd warshall was a very unnecessary idea, so here we go with the simple idea:
# part 1
galaxies = []
for i in range(image_expanded.shape[0]):
    for j in range(image_expanded.shape[1]):
        if image_expanded[i,j] == 1:
            galaxies.append((i,j))

distances = []
for i,g1 in enumerate(galaxies[:-1]):
    for g2 in galaxies[i+1:]:
        distances.append(abs(g1[0]-g2[0])+abs(g1[1]-g2[1]))

sum_distances = sum(distances)
print(f"Part 1: {sum_distances}")

Part 1: 10422930


In [53]:
#part 2
galaxies = []
for i in range(image.shape[0]):
    for j in range(image.shape[1]):
        if image[i,j] == 1:
            galaxies.append((i,j))

distances = []
for i,g1 in enumerate(galaxies[:-1]):
    for g2 in galaxies[i+1:]:
        (g1x,g1y) = g1
        (g2x,g2y) = g2
        #count row distance
        xdist = 0
        for i in range(min(g1x,g2x),max(g1x,g2x)):
            if np.sum(image[i,:]) == 0:
                xdist += 1e6
            else:
                xdist += 1
        ydist = 0
        for i in range(min(g1y,g2y),max(g1y,g2y)):
            if np.sum(image[:,i]) == 0:
                ydist += 1e6
            else:
                ydist +=1
        distances.append(ydist+xdist)

sum_distances = sum(distances)
print(f"Part 2: {sum_distances}")

Part 2: 699909023130.0
